In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#load data into dataframe
df= pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
df.shape
#df.head()
#df.columns

In [ ]:
#we can see that bmi has missing values,
#we will fill in with the mean
df['bmi'].fillna(value=df['bmi'].mean(), inplace=True)

In [ ]:
df_copy = df.copy()

y = df_copy['stroke']
X = df_copy.drop(['stroke'], axis=1)
X.columns



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

#it is important to get the categorical variables out so they can be one hot encoded
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
print(f'categorical variables: {object_cols}')



In [ ]:
numerical_cols = list(X_train.drop(object_cols, axis=1))
print(f'numerical variables: {numerical_cols}')

In [ ]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

In [ ]:
#OneHotEncoding removes the index. we will put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

In [ ]:
#The categorical values will be dropped to get the numerical values
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

In [ ]:
OH_X_train = pd.concat([OH_cols_train, num_X_train], axis=1)
OH_X_valid = pd.concat([OH_cols_valid, num_X_valid], axis=1)

In [ ]:
OH_X_valid.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[22]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='mae'
)

In [ ]:
history = model.fit(
    OH_X_train, y_train,
    validation_data=(OH_X_valid, y_valid),
    batch_size=256,
    epochs=11,
)

In [ ]:
y_preds = model.predict(OH_X_valid)

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_preds, y_valid)
print(mae)

In [ ]:
# convert the training history to a dataframe
history_df = pd.DataFrame(history.history)
# use Pandas native plot method
history_df['loss'].plot();